In [1]:
#随机更换代理，是反爬的trick之一，但是在实践过程中，并不work；
#更换header也是，但是从爬取的实践经验来看，time.sleep()的设置是非常重要的；
#sleep的时间设置长些，也就意味着访问url的间隔时间会长些，这样可以有效避免反爬；
import re
import requests
import urllib
import sys
!{sys.executable} -m pip install bs4
from bs4 import BeautifulSoup
import random
!{sys.executable} -m pip install pandas
import pandas as pd
import csv
from itertools import cycle
import time
import numpy as np
import random
!{sys.executable} -m pip install tqdm
import tqdm

def get_data(page, HEADER):
    #一页有30个帖子
    basic_url = 'https://www.douban.com/group/634189/discussion?start=%d'%(page*30)
    #basic_url = 'https://www.douban.com/group/638298/discussion?start=0'
    response = requests.get(basic_url, headers = HEADER, timeout= 60)
    html = response.text
#     print("html:", html)
    soup = BeautifulSoup(html, 'lxml')
    
    #获取帖子中的title,name,response,time;
    data = soup.find_all('tr')
    #filtered_data[i] = ['精品', 'title','name','response_nums','time'] or ['title','name','response_nums','time']
    filtered_data = [[i for i in i.text.strip().split('\n')if i.strip() != ''] for i in data[1:]]
    
    #获取帖子中的topic's url及发帖人id；
    topic_people_urls = [url.get('href') for url in soup.find_all('a')]
    topic_url = []
    people_url = []
    for topic_people_url in topic_people_urls:
        if topic_people_url is not None:
            topic_people_url_ = topic_people_url.split('/')
            #topic_url in one page;
            if len(topic_people_url_) < 3:
                continue
            if topic_people_url_[-3] == 'topic':
                topic_url.append(topic_people_url)
            #people_url in one page;
            elif topic_people_url_[-3] == 'people':
                people_url.append(topic_people_url)
    
             
    return filtered_data, topic_url, people_url

def get_most_popular_cont(soup):
    dataOfMostPopular = soup.find_all('ul', "topic-reply popular-bd")[0]

    #最赞回应的发帖方式（电脑orApp）；
    waysToPostsInMostPopular = []
    for i in dataOfMostPopular.find_all('li'):
        try:
            wayToPost = i.find_all('span', 'via')[0].text
        except:
            wayToPost = ''
        waysToPostsInMostPopular.append(wayToPost)

    #最赞回应的发帖内容；
    contsOfMostPopular = [i.text for i in dataOfMostPopular.find_all('p')]

    #最赞回应用户的nickname；
    nickNamesOfMostPopular = [i.text.split('\n')[1] for i in dataOfMostPopular.find_all('h4')]

    #最赞回应用户的id；
    idsOfMostPopular = [i.find('a').get('href') for i in dataOfMostPopular.find_all('h4')]
    
    return waysToPostsInMostPopular, contsOfMostPopular, nickNamesOfMostPopular, idsOfMostPopular


def get_way_id_name_cont(soup):
    data = soup.find_all('div', 'reply-doc content')
    #发帖方式（电脑orApp）
    ways = []
    for i in data:
        try:
            way = i.find_all('span', 'via')[0].text
        except:
            way = ''
        ways.append(way)

    #回复内容；
    conts = [j.text for i in data for j in i.find_all('p')]

    #回复人的nickname;
    nickNames = [i.find_all('a')[0].text for i in data]
    
    #回复人的id
    ids = [i.find_all('a')[0].get('href') for i in data]
    
    return ways, conts, nickNames, ids


def get_content(topic_url, custom):
    
    allWaysOfReplyContents = []
    allContsOfReplyContents = []
    allNicknamesOfReplyContents = []
    allIdsOfReplyContents = []
    allWaysOfReplyContents_ = []
    allContsOfReplyContents_ = []
    allNicknamesOfReplyContents_ = []
    allIdsOfReplyContents_ = []
    imgNumsInTopic_ = []
    topicDocs_ = []
    topicWays_ = []
    most_popular = []
    
    for url in topic_url:
        HEADER = {'User-Agent': random.choice(HEADERS_LIST)}
        response = requests.get(url, headers=HEADER, timeout= 60)
        html = response.text
        soup = BeautifulSoup(html, 'lxml')
        
        #帖子中包含图片的数量；
        try:
            imgNumsInTopic = len(soup.find_all('div', "rich-content topic-richtext")[0].find_all('img'))
        except:
            imgNumsInTopic = 0
        imgNumsInTopic_.append(imgNumsInTopic)
        
        topic = [i for i in soup.find_all('div','topic-doc')[0].text.split('\n') if i != '']
        
        try:
            topicDocs = ''.join([i.text for i in soup.find_all('div', "rich-content topic-richtext")[0].find_all('p')])
        except:
            topicDocs = '出现list_index错误'
        topicWays = topic[-1]
        
        topicDocs_.append(topicDocs)
        topicWays_.append(topicWays)
        
        if custom == 'mostPopular':
            #最赞回应的发帖方式（电脑orApp）、内容、昵称及id；
            waysToPostsInMostPopular, contsOfMostPopular, nickNamesOfMostPopular, idsOfMostPopular = get_most_popular_cont(soup)
            allWaysOfReplyContents_.append(waysToPostsInMostPopular)
            allContsOfReplyContents_.append(contsOfMostPopular)
            allNicknamesOfReplyContents_.append(nickNamesOfMostPopular)
            allIdsOfReplyContents_.append(idsOfMostPopular)
        
        elif custom == '1stPage':
            #第一页的发帖方式（电脑orApp）、内容、昵称及id；
            waysOfReplyIn1stPage, contsOfReplyIn1stPage, nickNamesOfReplyIn1stPage, idsOfReplyIn1stPage = get_way_id_name_cont(soup)
            try:
                len_popular, _, _, _ = get_most_popular_cont(soup)
                index = len(len_popular)
            except:
                index = 'none'
            allWaysOfReplyContents_.append(waysOfReplyIn1stPage)
            allContsOfReplyContents_.append(contsOfReplyIn1stPage)
            allNicknamesOfReplyContents_.append(nickNamesOfReplyIn1stPage)
            allIdsOfReplyContents_.append(idsOfReplyIn1stPage)
            most_popular.append(index)
            
        elif custom == 'allPages':
            #全部页的发帖方式（电脑orApp）、内容、昵称及id；
            waysOfReplyIn1stPage, contsOfReplyIn1stPage, nickNamesOfReplyIn1stPage, idsOfReplyIn1stPage = get_way_id_name_cont(soup)
            allWaysOfReplyContents.extend(waysOfReplyIn1stPage)
            allContsOfReplyContents.extend(contsOfReplyIn1stPage)
            allNicknamesOfReplyContents.extend(nickNamesOfReplyIn1stPage)
            allIdsOfReplyContents.extend(idsOfReplyIn1stPage)
            
            for page in allPagesOfReplyContents:
                response = requests.get(page, headers = HEADER, timeout= 60)
                htmlOfPage = response.text
                soupOfPage = BeautifulSoup(html, 'lxml')
                ways, conts, nickNames, ids = get_way_id_name_cont(soupOfPage)
                allWaysOfReplyContents.extend(ways)
                allContsOfReplyContents.extend(conts)
                allNicknamesOfReplyContents.extend(nickNames)
                allIdsOfReplyContents.extend(ids)
                time.sleep(3)
            
            allWaysOfReplyContents_.append(allWaysOfReplyContents)
            allContsOfReplyContents_.append(allContsOfReplyContents)
            allNicknamesOfReplyContents_.append(allNicknamesOfReplyContents)
            allIdsOfReplyContents_.append(allIdsOfReplyContents)
            
        time.sleep(3)
            
    return imgNumsInTopic_, topicDocs_, topicWays_, allWaysOfReplyContents_, allContsOfReplyContents_, allNicknamesOfReplyContents_, allIdsOfReplyContents_, most_popular

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [3]:
from tqdm import tqdm
import traceback
import logging
HEADERS_LIST = [
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
            "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
            "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
            "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
            "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
            "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
            "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
            "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
        ]

count = 1
for page in tqdm(range(0, 14)):
    
    if count % 18 == 0:
        time.sleep(random.randint(220, 230))
    
    if count % 37 == 0:
        time.sleep(random.randint(265, 280))
    
    if count % 56 == 0:
        time.sleep(random.randint(303, 336))
    
    HEADER = {'User-Agent': random.choice(HEADERS_LIST)}
    
    s_time = time.time()
    
    filtered_data, topic_url, people_url = get_data(page, HEADER)
    #topic_url = [i for i in topic_url if i.split(':')[0] != 'https']
    for i, j in enumerate(topic_url):
        if j.split(':')[0] != 'https':
            topic_url[i] = topic_url[-1]
            
    time.sleep(5)
    
    imgNumsInTopic_, topicDocs_, topicWays_, allWaysOfReplyContents_, allContsOfReplyContents_, allNicknamesOfReplyContents_, allIdsOfReplyContents_, most_popular = get_content(topic_url, '1stPage')
    print(topicDocs_)
    
    jingpin = []
    for i in filtered_data:
        try:
            jingpin.append(i[-5])
        except:
            jingpin.append('')
    
    try:
        titles = [i[-4] for i in filtered_data]
    except:
        titles = [i[0] for i in filtered_data]
    try:
        names = [i[-3] for i in filtered_data]
        responses_num = [i[-2] for i in filtered_data]
        times = [i[-1] for i in filtered_data]

        #if len(allWaysOfReplyContents_) != len(titles):
         #   allWaysOfReplyContents_.append(allWaysOfReplyContents_[-1])

        csv = pd.DataFrame({'title':titles, 'name':names, 'id':people_url,
                            'responses_num':responses_num, 'time':times, 'docs':topicDocs_,
                            'reply_content':allContsOfReplyContents_, 'img_nums':imgNumsInTopic_, 
                            'topic_url':topic_url, 'reply_id':allIdsOfReplyContents_, 'jinpin':jingpin, 
                            'popular_index':most_popular, 'topic_ways':topicWays_, 
                            'replyWays':allWaysOfReplyContents_})

        csv.to_csv(r'test.csv', mode= 'a', header= True, encoding= 'utf_8_sig')
        time.sleep(random.randrange(6, 8, 1))
        e_time = time.time()
        log = '第%d页'%(page+1) + '||' + '发帖时间：%s'%times[-1] + '||' + '耗时：%f秒'%(e_time-s_time)
        print(log)
        count = count + 1
    except IOError:
        print("I/O error")
    except IndexError:
        logging.error(traceback.format_exc())
    except Exception as e:
        logging.error(traceback.format_exc())
        raise

  0%|          | 0/14 [00:00<?, ?it/s]

['那个LZ什么锤都拿不出还非要数着日子爆料，反复开了几个贴说辞模棱两可。蒸煮迅速下场上热搜，搞事情的LZ怂成狗立马注销。本来完全不知道他演了《乔家的儿女》，现在又是编剧澄清狗血剧情，又是演员澄清婚变，这剧立马就有存在感了。是给剧炒热度撕暑假档吗？月底前卫视暑期排片就要出来了，其他片方都加油啊！炒起来！', '总是看到这种论调，十有八九还是为了讽刺杨洋可这明明杨洋的能力，是粉丝的荣光，怎么会觉得这样能黑到杨洋？杨洋演的大ip，除了全职男主，其他哪个不是高颜值and清冷气质设定，说杨洋演的这些大ip能奶几个生的，先看看你想奶的这些生有没有杨洋帅，颜值气质配不配得上这些小说人物的设定，作者能不能看得上。先说盗墓笔记，小透明重新出现在公众视野就是饰演燃王张起灵，当初杨洋最开始试镜的只是配角小花，是作者自己选他演小哥的再说左耳，饶雪漫亲口说他们看了几乎全中国的漂亮男孩，但是都没有满意的，天中的风云人物校草许弋一定要非常帅，直到看到了杨洋，她和苏有朋导演都非常激动，觉得杨洋就是许弋，非他演不可微微一笑很倾城，不用多说了吧，顾漫喜欢杨洋以上皆有作者视频音频为证全职高手，杨洋人气最高的时候接的，当年的鹅还不是现在财大气粗的鹅，一个除了女主全员小透明的剧，和这剧打包的还有鹅自家的5个代言，还用多说吗？没红前作者喜欢，作品奶杨洋，红了以后平台喜欢，相互成就，谁也不亏。', '郑恺问蔡徐坤包几场电影， 坤子说取决于你买几张专辑，你买几张我包几场，谁能想到恺哥直接上来就是按头100场，坤子后面算着不对劲，赶紧说你得买我1000张专辑视频 - 微博 t.cn                                                     可是坤子你不知道你还是亏大了么，你以为包一场是买一张电影票么！坤子，以后算数的事还是交给精打细算的ikun吧，保证稳赚不赔！不知道这娃现在知不知道自己亏大发了！😂😂😂', '或比较推荐什么牌子的卫生巾我最近用了比较火的液体卫生巾，但感觉一般般', '认识罗正是在偶练，他唱跳实力不ok，而且当时因为没自信整个人非常丧，因为脸蛋苟了好一阵子，最终还是实力不行合理淘汰。但是！！！我的帅哥雷达整天滴滴滴为他作响啊！！长相疯狂踩我的审美点！！！出厂以后，他没什么资源，我也有追的人很长一段时间没怎么关注他动态，但微博关注一直为这个帅哥留着，昨天

  7%|▋         | 1/14 [02:30<32:38, 150.63s/it]

第1页||发帖时间：06-09 16:51||耗时：150.634511秒
['或比较推荐什么牌子的卫生巾我最近用了比较火的液体卫生巾，但感觉一般般', '。。。Emmmm，超哥，对不住————————————————————粉丝提名区补充:', '鉴于龚俊的微博管理得很好，早年的微博现在已经删得不剩太多了，接到投稿，分享一下。他以前...真的好屌丝....简直和二舅土得不分上下...早年站姐出图水平也堪忧...只能说ylq真是人要衣装佛要金装...狠狠包装一下什么男神都出来了……', '马伊琍好多件旗袍好美！还有洋裙，好美！高伟光好多件大衣好帅！还有衬衫马甲，也帅！剧情emmm…反正破案部分我…觉得无所谓了……我主要是看衣服和脸😂😂😂马伊琍演这种有点作有点嗲又很有主意的角色真的不错子，虽然有好多人说做作啥的，但我觉得人设就是这个亚子😂恰到好处~而且身材🤤🤤🤤🤤🤤🤤我那段时间踩椭圆机的时候特别喜欢看，踩个1集长度，结束~其实里面董璇穿得也还行，但发型实在有点失败，显得脸有点大，我就不放图了😳', '后续来了：（不知道是不是P图但是太好笑了！！！）', '抽一个人告诉我  这个人结局会不会砸电视 ', '我们就是个吃瓜组，也不知道啥真🍉假🍉，爆料帖不代表本组立场！不造谣，不传谣，发现假🍉，就地正法！爆料原帖楼主已销号，帖子本人已删除：', '首先说一下我自己，我在混粉圈之前，对于粉丝买明星代言的手机都是嗤之以鼻，粉丝就是被割韭菜的傻逼。如果有例外，可以上配置图来打我的脸，要有性价比。就算我现在混粉圈我也不会想买明星代言的手机，除非配置和性价比都🉑。最近想换手机了！有没有粉丝过来推销！我去给你们贡献销量。', '谢谢组长放我进来！为我组KPI狠狠努力！！——————以下正文———————先来几个Q&AQ1：戴牙套会有牙套脸吗？A：因人而异，牙套脸主要是戴牙套咀嚼能力下降咬肌变小，导致视觉上的颧骨变大变宽的样子，如果带了牙套还能和我一样吃每日坚果那你就不会。Q2：戴牙套要拔牙吗？A：因人而异，看突度和需要内收的空间大小。本人很幸运拔了四颗4号牙🤪🤪🤪Q3：戴牙套疼吗？A：疼，每次复诊都会疼，但在可以接受的范围内。原则下牙比上牙疼，下牙神经比上牙多。Q4：戴牙套会瘦吗？A：因人而异。比如我戴了牙套依然可以大吃四方，体重依然没变。但是很多人因为怕吃东西刷牙，所以放弃吃东西导致

 14%|█▍        | 2/14 [04:54<29:19, 146.66s/it]

...

第7页||发帖时间：06-09 16:37||耗时：141.855413秒
['八集，应该没有什么要注意的，我感觉这个剧在咱们组讨论度也挺低的，蛮糊。在b站看了一小段第一集cut，太刺激了，垂直入坑。没有要注意的，这栋楼就当我追完剧给大家写repo的楼吧。', '人都是会老的，每个年龄段都有每个年龄段的魅力那才是最棒的，现在的帅哥肖战看腻了叭，看看爱豆时期的肖战叭（毕竟谁还不是个爱豆了）他真是什么时候都好看，都说“岁月是把杀猪刀”，但还好只杀猪，不杀兔子！生图🈶，不过不多加几张最近的图再来几张动图6.7更新：时影大神官（神图太多，一个动作只发一张）', '看看荣耀和且试天下的对比好讽刺啊……说迪丽热巴家撕番的可以看看👇🏻【划重点:巴粉同意全平台都让潘老师第一，按着官宣顺序排】这种程度了，真的只是这是合理维权吧？说迪丽热巴撕番的不知道怎么想的团队沟通全过程➕合约@顶书过河 的微博: 荣耀这部剧的情况全网皆知，本无可争议。巴... weibo.cn🟥没有争一番，而是一直强调【平番】，甚至把愿意按年龄排序让潘老师在平台第一位。我真的觉得大美女粉丝这次没啥错…所以到底谁带头撕番……巴粉有实力维权能控评导致前排全是巴粉，所以就是巴粉撕番？？这波操作真好……果然什么都不会只会哭的孩子有糖吃~这次我就为大美女巴巴站队了~', '李佳航和李晟俩人从《新版还珠格格》认识，因戏生情。两人2014年公开恋情。2015年结婚。结婚照片结婚三年后李晟怀孕，下面是孕期照片。迎来宝贝儿子李品序。下面更新两人微博秀恩爱的日常更新两人情侣艺术照', '我平等地歌唱女性和男性。', '【任嘉伦|刘亦菲|陆绎|小龙女】神雕侠侣AU|神话情话|feat. 老神宣_哔哩哔哩_bilibili', '万万没想到竟然考了国歌的歌词来答题 ', '这是2月份抖音里的米卡。。【抖音】记录美好生活 douyin.com这是现在的米卡这个感觉怎么变了呢。。。几个月不见你咋变了呢。。岭做的最好的事大概就是让我看了创4两期后上头了岭弃了创4了，不然我如果zqsg追了，最后团变成这样，一口老血。。顿时原谅了岭～现在的成团的几位和我之前看创的时候的感觉都不一样了。。', '…', '首先耽改已经很多人拍了各种各样的都有，但固定人群喜欢这类的也就部分，能🔥但难大爆同时期竞争也挺激烈的容易产生多担，最后都跟选秀疲劳一样，一个剧播的

 57%|█████▋    | 8/14 [18:59<14:08, 141.47s/it]

第8页||发帖时间：06-09 16:31||耗时：141.726844秒


 57%|█████▋    | 8/14 [19:49<14:52, 148.71s/it]


KeyboardInterrupt: 